In [2356]:
import torch
import torch.nn as nn
import pandas as pd

In [2357]:
gender_df = pd.read_csv('gender_submission.csv')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [2358]:
X_train = train_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked', 'Fare']]
X_test = test_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked', 'Fare']]
y_train = train_df['Survived'].values

mean_age = X_train['Age'].describe()['75%']
X_train['Age'].fillna(mean_age, inplace=True)
X_test['Age'].fillna(mean_age, inplace=True)

X_train['Sex'].replace(['male', 'female'], [0,1], inplace=True)
X_test['Sex'].replace(['male', 'female'], [0,1], inplace=True)

top_embarked = X_train['Embarked'].describe().top
X_train['Embarked'].fillna(top_embarked, inplace=True)
X_test['Embarked'].fillna(top_embarked, inplace=True)

X_train['Embarked'].replace(['S', 'Q', 'C'], [0, 1, 2], inplace=True)
X_test['Embarked'].replace(['S', 'Q', 'C'], [0, 1, 2], inplace=True)

mean_fare = X_train['Fare'].mean()
X_train['Fare'].fillna(mean_fare, inplace=True)
X_test['Fare'].fillna(mean_fare, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [2359]:
class TitanicRegressor(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear_layer1 = nn.Linear(in_features=7, out_features=64)
    self.linear_layer2 = nn.Linear(in_features=64, out_features=32)
    self.linear_layer3 = nn.Linear(in_features=32, out_features=12)
    self.linear_layer4 = nn.Linear(in_features=12, out_features=1)
    self.relu = nn.ReLU()
    self.lrelu = nn.LeakyReLU()
    self.tanh = nn.Tanh()
    self.activation = nn.Sigmoid()
  
  def forward(self, x):
    x = self.tanh(self.linear_layer1(x))
    x = self.tanh(self.linear_layer2(x))
    x = self.relu(self.linear_layer3(x))
    x = self.linear_layer4(x)
    return self.activation(x)

model = TitanicRegressor()

In [2360]:
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)


In [2361]:
X_train = nn.functional.normalize(X_train)
X_test = nn.functional.normalize(X_test)

In [2362]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
epochs = 1000

In [2363]:
for epoch in range(epochs):
  optimizer.zero_grad()
  preds = model(X_train)
  loss = loss_fn(preds, y_train.unsqueeze(1))
  if epoch % 100 == 0:
    print(f"Epoch: {epoch}, Loss: {loss}")
  
  
  loss.backward()
  optimizer.step()

Epoch: 0, Loss: 0.7057760953903198
Epoch: 100, Loss: 0.4278724789619446
Epoch: 200, Loss: 0.39264655113220215
Epoch: 300, Loss: 0.36627545952796936
Epoch: 400, Loss: 0.34374698996543884
Epoch: 500, Loss: 0.3310019373893738
Epoch: 600, Loss: 0.3313558101654053
Epoch: 700, Loss: 0.28394457697868347
Epoch: 800, Loss: 0.315529465675354
Epoch: 900, Loss: 0.24122385680675507


In [2364]:
model.eval()
test_preds = torch.round(model(X_test))

In [2368]:
with torch.no_grad():
  test_df['Survived'] = pd.Series(test_preds.squeeze(1).numpy().astype(int))

test_df['Survived'].values.sum()

168

In [2366]:
test_df[['PassengerId','Survived']].to_csv('submission.csv', index=False)

In [2367]:
df = pd.read_csv('submission.csv')
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
